# LSTM implementation for TORCS driver

In [1]:
import glob
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import Imputer
from collections import defaultdict

## LSTM Definition

In [2]:
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, batch_size):
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        
        super(RNN_LSTM, self).__init__()        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.out = nn.Linear(hidden_size, output_size)        
        self.hidden = self.init_hidden()
        
    def init_hidden(self, x=None):
        if x == None:
            return (Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size)))
        else:
            return (Variable(x[0].data),Variable(x[1].data))
        
    def forward(self, x):
        lstm_out, hidden_out = self.lstm(x, self.hidden)
        output = self.out(lstm_out.view(len(x), -1))
        self.hidden = self.init_hidden(hidden_out)
        return output

## Train the network

The LSTM has 28 inputs, 3 hidden layers with 28 hidden units, and an output layer with 3 nodes. The batch size is set to 100, while the learning rate is variable along the 500 epochs.

In [52]:
EPOCHS_LR = {
    0.9: 10,
    0.7: 11,
    0.5: 15,
    0.3: 20,
    0.1: 25,
    0.01: 30,
    0.005: 40,
    0.001: 50,
    0.0005: 50,
    0.0001: 50
}

In [64]:
INPUT_SIZE = 28
HIDDEN_SIZE = 28
NUM_LAYERS = 3
BATCH_SIZE = 50
NUM_EPOCHS = 20
LEARNING_RATE = 0.0001
# LRS = np.concatenate((np.arange(0.9, 0.1, -0.1),
#                       [0.15, 0.1, 0.05, 0.01, 0.0075, 0.005, 0.0025, 0.001, 0.00075, 0.0005, 0.00025, 0.0001]))
LRS = np.concatenate((np.arange(0.9, 0, -0.2),
                      [0.01, 0.005, 0.001, 0.0005, 0.0001]))

lstm_nn = RNN_LSTM(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, 3, BATCH_SIZE)
criterion = nn.MSELoss()

In [65]:
def normalize(x, mmin, mmax):
    return (x - mmin)/(mmax-mmin)

# Get training filenames
training_files = glob.glob('train_data/*.csv')

# Search min and max values for specific parameters
maxSpeedX = -10000
minSpeedX = 10000
maxSpeedY = -10000
minSpeedY = 10000
maxRPM = 0
maxWheelSpin = 0

for f in training_files:
    train_ds = pd.read_csv(f)    
    X = train_ds.iloc[:, :-4].values
    
    if X[0].max() > maxSpeedX:
        maxSpeedX = X[0].max()
    if X[0].min() < minSpeedX:
        minSpeedX = X[0].min()
        
    if X[1].max() > maxSpeedY:
        maxSpeedY = X[1].max()
    if X[1].min() < minSpeedY:
        minSpeedY = X[1].min()    
    
    if X[4].max() > maxRPM:
        maxRPM = X[4].max()
    
    if X[5].max() > maxWheelSpin:
        maxWheelSpin = X[5].max()
    if X[6].max() > maxWheelSpin:
        maxWheelSpin = X[6].max()
    if X[7].max() > maxWheelSpin:
        maxWheelSpin = X[7].max()
    if X[8].max() > maxWheelSpin:
        maxWheelSpin = X[8].max()

# Save their values
param_dict = {
    'maxSpeedX':maxSpeedX,
    'minSpeedX':minSpeedX,
    'maxSpeedY':maxSpeedY,
    'minSpeedY':minSpeedY,
    'maxRPM':maxRPM,
    'maxWheelSpin':maxWheelSpin
}
with open('norm_parameters.pickle', 'wb') as handle:
    pickle.dump(param_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Read all training sets
training_sets = defaultdict(lambda: dict())

for f in training_files:
    # read dataset
    train_ds = pd.read_csv(f, header=None)
    
    X = train_ds.iloc[:, :-4].values
    y = train_ds.iloc[:, -4:].values
    
    # fill missing values with mean
    imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imputer = imputer.fit(X)
    X = imputer.transform(X)
    
    # normalize all values for interval [0, 1]    
    X[0] = normalize(X[0], minSpeedX, maxSpeedX)  # speedX = range(search min, search max)
    X[1] = normalize(X[1], minSpeedY, maxSpeedY)  # speedY = range(search min, search max)
    X[2] = normalize(X[2], -180, 180)  # angle = range(-180, 180)
    X[3] = normalize(X[3], -1, 6)  # currentGear = range(-1, 6)
    X[4] = normalize(X[4], 0, maxRPM)  # RPM = range(0, search max)
    for i in np.arange(5, 9):
        X[i] = normalize(X[i], 0, maxWheelSpin)  # *wheelSpin = range(0, search max)
    for i in np.arange(9, 28):
        X[i] = normalize(X[i], 0, 200)  # *sensorValues = range(0, 200)
    y[0] = normalize(y[0], -1, 6)  # gear = range(-1, 6)
    y[1] = normalize(y[1], -1, 1)  # steering = range(-1, 1)
    # for acceleration and break, compute their difference and normalize it
    accel_brake = y[2] - y[3]
    y[2] = normalize(accel_brake, -1, 1)  # accelerate-brake = range(-1, 1)
    y = np.delete(y, 3, axis=1)
    
    # Create TensorDataset from FloatTensors and save to dictionary
    X_train = torch.from_numpy(X).float()
    y_train = torch.from_numpy(y).float()
    dataset = TensorDataset(X_train, y_train)
    training_sets[f] = dataset

In [39]:
# Train with changing learning rate on number of epochs
for lr in EPOCHS_LR:
    no_epochs = EPOCHS_LR[lr]
    print('learning rate %.4f' %(lr))

    optimizer = torch.optim.Adam(lstm_nn.parameters(), lr=lr)
    
    for epoch in np.arange(no_epochs):
        for f in training_files:
            train_loader = DataLoader(dataset=training_sets[f], batch_size=BATCH_SIZE, shuffle=False)    
            lstm_nn.init_hidden()

            for i, (X, y) in enumerate(train_loader):
                if (len(X) != BATCH_SIZE):
                    continue

                data = Variable(X.view(-1, 1, INPUT_SIZE))
                target = Variable(y)

                optimizer.zero_grad()
                prediction = lstm_nn(data)
                loss = criterion(prediction, target)
                loss.backward()
                optimizer.step()

                if (i+1) % BATCH_SIZE == 0:
                    print('  epoch: [%d/%d], step: [%d/%d], loss: %.4f'
                          %(epoch+1, no_epochs, i+1, len(training_sets[f].target_tensor)//BATCH_SIZE, loss.data[0]))


learning rate 0.9000
  epoch: [1/10], step: [100/116], loss: 0.9781
  epoch: [1/10], step: [100/231], loss: 2.4053
  epoch: [1/10], step: [200/231], loss: 0.4660
  epoch: [1/10], step: [100/118], loss: 0.6651
  epoch: [2/10], step: [100/116], loss: 0.9013
  epoch: [2/10], step: [100/231], loss: 2.4730
  epoch: [2/10], step: [200/231], loss: 0.5272
  epoch: [2/10], step: [100/118], loss: 0.7848
  epoch: [3/10], step: [100/116], loss: 0.9126
  epoch: [3/10], step: [100/231], loss: 2.4304
  epoch: [3/10], step: [200/231], loss: 0.5410
  epoch: [3/10], step: [100/118], loss: 0.8292
  epoch: [4/10], step: [100/116], loss: 0.9217
  epoch: [4/10], step: [100/231], loss: 2.4125
  epoch: [4/10], step: [200/231], loss: 0.5449
  epoch: [4/10], step: [100/118], loss: 0.8476
  epoch: [5/10], step: [100/116], loss: 0.9255
  epoch: [5/10], step: [100/231], loss: 2.4032
  epoch: [5/10], step: [200/231], loss: 0.5433
  epoch: [5/10], step: [100/118], loss: 0.8553
  epoch: [6/10], step: [100/116], loss:

  epoch: [8/20], step: [100/116], loss: 1.2347
  epoch: [8/20], step: [100/231], loss: 1.2690
  epoch: [8/20], step: [200/231], loss: 0.2810
  epoch: [8/20], step: [100/118], loss: 0.6218
  epoch: [9/20], step: [100/116], loss: 1.2346
  epoch: [9/20], step: [100/231], loss: 1.2690
  epoch: [9/20], step: [200/231], loss: 0.2809
  epoch: [9/20], step: [100/118], loss: 0.6218
  epoch: [10/20], step: [100/116], loss: 1.2346
  epoch: [10/20], step: [100/231], loss: 1.2690
  epoch: [10/20], step: [200/231], loss: 0.2809
  epoch: [10/20], step: [100/118], loss: 0.6218
  epoch: [11/20], step: [100/116], loss: 1.2346
  epoch: [11/20], step: [100/231], loss: 1.2690
  epoch: [11/20], step: [200/231], loss: 0.2809
  epoch: [11/20], step: [100/118], loss: 0.6218
  epoch: [12/20], step: [100/116], loss: 1.2346
  epoch: [12/20], step: [100/231], loss: 1.2690
  epoch: [12/20], step: [200/231], loss: 0.2809
  epoch: [12/20], step: [100/118], loss: 0.6218
  epoch: [13/20], step: [100/116], loss: 1.2346


  epoch: [6/30], step: [100/116], loss: 0.9420
  epoch: [6/30], step: [100/231], loss: 0.9023
  epoch: [6/30], step: [200/231], loss: 0.2066
  epoch: [6/30], step: [100/118], loss: 0.4286
  epoch: [7/30], step: [100/116], loss: 0.9420
  epoch: [7/30], step: [100/231], loss: 0.9023
  epoch: [7/30], step: [200/231], loss: 0.2066
  epoch: [7/30], step: [100/118], loss: 0.4285
  epoch: [8/30], step: [100/116], loss: 0.9420
  epoch: [8/30], step: [100/231], loss: 0.9023
  epoch: [8/30], step: [200/231], loss: 0.2066
  epoch: [8/30], step: [100/118], loss: 0.4285
  epoch: [9/30], step: [100/116], loss: 0.9420
  epoch: [9/30], step: [100/231], loss: 0.9023
  epoch: [9/30], step: [200/231], loss: 0.2066
  epoch: [9/30], step: [100/118], loss: 0.4285
  epoch: [10/30], step: [100/116], loss: 0.9420
  epoch: [10/30], step: [100/231], loss: 0.9023
  epoch: [10/30], step: [200/231], loss: 0.2066
  epoch: [10/30], step: [100/118], loss: 0.4285
  epoch: [11/30], step: [100/116], loss: 0.9420
  epoch:

  epoch: [19/40], step: [100/116], loss: 0.9496
  epoch: [19/40], step: [100/231], loss: 0.8840
  epoch: [19/40], step: [200/231], loss: 0.2187
  epoch: [19/40], step: [100/118], loss: 0.4738
  epoch: [20/40], step: [100/116], loss: 0.9496
  epoch: [20/40], step: [100/231], loss: 0.8840
  epoch: [20/40], step: [200/231], loss: 0.2187
  epoch: [20/40], step: [100/118], loss: 0.4738
  epoch: [21/40], step: [100/116], loss: 0.9496
  epoch: [21/40], step: [100/231], loss: 0.8840
  epoch: [21/40], step: [200/231], loss: 0.2187
  epoch: [21/40], step: [100/118], loss: 0.4738
  epoch: [22/40], step: [100/116], loss: 0.9496
  epoch: [22/40], step: [100/231], loss: 0.8840
  epoch: [22/40], step: [200/231], loss: 0.2187
  epoch: [22/40], step: [100/118], loss: 0.4738
  epoch: [23/40], step: [100/116], loss: 0.9496
  epoch: [23/40], step: [100/231], loss: 0.8840
  epoch: [23/40], step: [200/231], loss: 0.2187
  epoch: [23/40], step: [100/118], loss: 0.4738
  epoch: [24/40], step: [100/116], loss:

  epoch: [21/50], step: [100/118], loss: 0.5458
  epoch: [22/50], step: [100/116], loss: 0.9728
  epoch: [22/50], step: [100/231], loss: 0.8645
  epoch: [22/50], step: [200/231], loss: 0.2266
  epoch: [22/50], step: [100/118], loss: 0.5458
  epoch: [23/50], step: [100/116], loss: 0.9728
  epoch: [23/50], step: [100/231], loss: 0.8645
  epoch: [23/50], step: [200/231], loss: 0.2266
  epoch: [23/50], step: [100/118], loss: 0.5458
  epoch: [24/50], step: [100/116], loss: 0.9728
  epoch: [24/50], step: [100/231], loss: 0.8645
  epoch: [24/50], step: [200/231], loss: 0.2266
  epoch: [24/50], step: [100/118], loss: 0.5458
  epoch: [25/50], step: [100/116], loss: 0.9728
  epoch: [25/50], step: [100/231], loss: 0.8645
  epoch: [25/50], step: [200/231], loss: 0.2266
  epoch: [25/50], step: [100/118], loss: 0.5458
  epoch: [26/50], step: [100/116], loss: 0.9728
  epoch: [26/50], step: [100/231], loss: 0.8645
  epoch: [26/50], step: [200/231], loss: 0.2266
  epoch: [26/50], step: [100/118], loss:

  epoch: [14/50], step: [200/231], loss: 0.2264
  epoch: [14/50], step: [100/118], loss: 0.5561
  epoch: [15/50], step: [100/116], loss: 0.9768
  epoch: [15/50], step: [100/231], loss: 0.8624
  epoch: [15/50], step: [200/231], loss: 0.2264
  epoch: [15/50], step: [100/118], loss: 0.5561
  epoch: [16/50], step: [100/116], loss: 0.9768
  epoch: [16/50], step: [100/231], loss: 0.8624
  epoch: [16/50], step: [200/231], loss: 0.2264
  epoch: [16/50], step: [100/118], loss: 0.5561
  epoch: [17/50], step: [100/116], loss: 0.9768
  epoch: [17/50], step: [100/231], loss: 0.8624
  epoch: [17/50], step: [200/231], loss: 0.2264
  epoch: [17/50], step: [100/118], loss: 0.5561
  epoch: [18/50], step: [100/116], loss: 0.9768
  epoch: [18/50], step: [100/231], loss: 0.8624
  epoch: [18/50], step: [200/231], loss: 0.2264
  epoch: [18/50], step: [100/118], loss: 0.5561
  epoch: [19/50], step: [100/116], loss: 0.9768
  epoch: [19/50], step: [100/231], loss: 0.8624
  epoch: [19/50], step: [200/231], loss:

  epoch: [7/50], step: [100/231], loss: 0.8606
  epoch: [7/50], step: [200/231], loss: 0.2260
  epoch: [7/50], step: [100/118], loss: 0.5643
  epoch: [8/50], step: [100/116], loss: 0.9798
  epoch: [8/50], step: [100/231], loss: 0.8607
  epoch: [8/50], step: [200/231], loss: 0.2260
  epoch: [8/50], step: [100/118], loss: 0.5643
  epoch: [9/50], step: [100/116], loss: 0.9799
  epoch: [9/50], step: [100/231], loss: 0.8607
  epoch: [9/50], step: [200/231], loss: 0.2260
  epoch: [9/50], step: [100/118], loss: 0.5643
  epoch: [10/50], step: [100/116], loss: 0.9799
  epoch: [10/50], step: [100/231], loss: 0.8607
  epoch: [10/50], step: [200/231], loss: 0.2260
  epoch: [10/50], step: [100/118], loss: 0.5642
  epoch: [11/50], step: [100/116], loss: 0.9799
  epoch: [11/50], step: [100/231], loss: 0.8607
  epoch: [11/50], step: [200/231], loss: 0.2260
  epoch: [11/50], step: [100/118], loss: 0.5642
  epoch: [12/50], step: [100/116], loss: 0.9799
  epoch: [12/50], step: [100/231], loss: 0.8607
  e

  epoch: [50/50], step: [100/116], loss: 0.9801
  epoch: [50/50], step: [100/231], loss: 0.8609
  epoch: [50/50], step: [200/231], loss: 0.2259
  epoch: [50/50], step: [100/118], loss: 0.5642


In [17]:
# Changing learning rate
lridx = -1

epoch = 0

while epoch < NUM_EPOCHS:
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))
    
    if epoch % 50 == 0:
        lridx += 1
        optimizer = torch.optim.Adam(lstm_nn.parameters(), lr=LRS[lridx])
    
    for f in training_files:
#         print('  training set: %s' %(f[f.find('/')+1:]))        
        train_loader = DataLoader(dataset=training_sets[f], batch_size=BATCH_SIZE, shuffle=False)    
        lstm_nn.init_hidden()

        for i, (X, y) in enumerate(train_loader):
            if (len(X) != BATCH_SIZE):
                continue

            data = Variable(X.view(-1, 1, INPUT_SIZE))
            target = Variable(y)

            optimizer.zero_grad()
            prediction = lstm_nn(data)
            loss = criterion(prediction, target)
            loss.backward()
            optimizer.step()

            if (i+1) % BATCH_SIZE == 0:
                
                print('    step: [%d/%d], loss: %.4f'
                      %(i+1, len(training_sets[f].target_tensor)//BATCH_SIZE, loss.data[0]))

print('Training done')

Epoch [1/500]
    step: [100/116], loss: 2.5588
    step: [100/231], loss: 1.0885
    step: [200/231], loss: 0.8383
    step: [100/118], loss: 0.8573
Epoch [1/500]
    step: [100/116], loss: 2.8224
    step: [100/231], loss: 0.9594
    step: [200/231], loss: 0.7714
    step: [100/118], loss: 1.2330


KeyboardInterrupt: 

In [66]:
# Same learning rate
optimizer = torch.optim.Adam(lstm_nn.parameters(), lr=LEARNING_RATE)
for epoch in np.arange(NUM_EPOCHS):
    print('Epoch [%d/%d]' %(epoch+1, NUM_EPOCHS))    
    for f in training_files:
        train_loader = DataLoader(dataset=training_sets[f], batch_size=BATCH_SIZE, shuffle=False)    
        lstm_nn.init_hidden()

        for i, (X, y) in enumerate(train_loader):
            if (len(X) != BATCH_SIZE):
                continue

            data = Variable(X.view(-1, 1, INPUT_SIZE))
            target = Variable(y)

            optimizer.zero_grad()
            prediction = lstm_nn(data)
            loss = criterion(prediction, target)
            loss.backward()
            optimizer.step()

            if (i+1) % BATCH_SIZE == 0:                
                print('    step: [%d/%d], loss: %.4f'
                      %(i+1, len(training_sets[f].target_tensor)//BATCH_SIZE, loss.data[0]))

print('Training done')

Epoch [1/20]
    step: [50/142], loss: 3.1737
    step: [100/142], loss: 5.7125
    step: [50/232], loss: 1.2703
    step: [100/232], loss: 1.4347
    step: [150/232], loss: 0.1856
    step: [200/232], loss: 0.3470
    step: [50/462], loss: 0.5526
    step: [100/462], loss: 0.1797
    step: [150/462], loss: 1.8688
    step: [200/462], loss: 0.4442
    step: [250/462], loss: 0.2204
    step: [300/462], loss: 1.7494
    step: [350/462], loss: 1.0708
    step: [400/462], loss: 0.1297
    step: [450/462], loss: 0.6466
    step: [50/237], loss: 0.9451
    step: [100/237], loss: 0.1667
    step: [150/237], loss: 0.1401
    step: [200/237], loss: 0.2873
    step: [50/198], loss: 0.0658
    step: [100/198], loss: 0.0562
    step: [150/198], loss: 0.0739
Epoch [2/20]
    step: [50/142], loss: 0.0370
    step: [100/142], loss: 1.7876
    step: [50/232], loss: 0.1572
    step: [100/232], loss: 0.3755
    step: [150/232], loss: 0.0637
    step: [200/232], loss: 0.2741
    step: [50/462], loss: 0.2

    step: [50/198], loss: 0.3393
    step: [100/198], loss: 0.0169
    step: [150/198], loss: 0.1122
Epoch [12/20]
    step: [50/142], loss: 0.0499
    step: [100/142], loss: 1.4946
    step: [50/232], loss: 0.2011
    step: [100/232], loss: 0.0145
    step: [150/232], loss: 0.1198
    step: [200/232], loss: 0.3255
    step: [50/462], loss: 0.0890
    step: [100/462], loss: 0.0759
    step: [150/462], loss: 0.2108
    step: [200/462], loss: 0.4438
    step: [250/462], loss: 0.0279
    step: [300/462], loss: 0.2133
    step: [350/462], loss: 0.6448
    step: [400/462], loss: 0.1940
    step: [450/462], loss: 0.0098
    step: [50/237], loss: 0.1174
    step: [100/237], loss: 0.1301
    step: [150/237], loss: 0.1149
    step: [200/237], loss: 0.1779
    step: [50/198], loss: 0.3384
    step: [100/198], loss: 0.0411
    step: [150/198], loss: 0.0902
Epoch [13/20]
    step: [50/142], loss: 0.0515
    step: [100/142], loss: 1.4561
    step: [50/232], loss: 0.2300
    step: [100/232], loss: 0

## Save model parameters

Parameters will be used by the driver to get a command to the server.

In [67]:
torch.save(lstm_nn.state_dict(), 'rnn_params.pt')

In [44]:
torch.save(lstm_nn, 'whole_net.pt')

/usr/local/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type RNN_LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Backup code

In [ ]:
# for file in training_files:
#     # Train the network for each training session
#     print('Training file: %s' %(file))
    
#     train_ds = pd.read_csv(file)
#     X_train = train_ds.iloc[:, 3:].values
#     y_train = train_ds.iloc[:, :3].values
    
#     imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
#     imputer = imputer.fit(X_train)
#     X_train = imputer.transform(X_train)
    
#     X_train = torch.from_numpy(X_train).float()
#     y_train = torch.from_numpy(y_train).float()
    
#     dataset = TensorDataset(X_train, y_train)
    
#     train_loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     rnn.init_hidden()
    
#     for epoch in range(NUM_EPOCHS):
#         for i, (X, y) in enumerate(train_loader):
#             if (len(X) != BATCH_SIZE):
#                 continue
            
#             data = Variable(X.view(-1, 1, INPUT_SIZE))
#             target = Variable(y)
            
#             optimizer.zero_grad()
#             prediction = rnn(data)
#             loss = criterion(prediction, target)
#             loss.backward()
#             optimizer.step()
            
#             if (i+1) % 30 == 0:
#                 print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
#                       %(epoch+1, NUM_EPOCHS, i+1, len(X_train)//BATCH_SIZE, loss.data[0]))